In [1]:
%load_ext autoreload
%autoreload 2


# Load relevent data

rosters

processed possession data

In [2]:
from nba import NbaTracker
from nba_api.stats.static.players import find_player_by_id
from nba_api.stats.static.teams import find_team_name_by_id

nbaTracker = NbaTracker()
season_name = "2015-16"
nbaTracker.add_season(season_name)
season = nbaTracker.seasons[season_name]
season.load_possession_data()
season.build_player_seasons()
games = season.games


# discover rebound data

In [3]:
from build_shot_chance_data import EventType
from collections import defaultdict

rb_result_counts = defaultdict(int)
rb_prev_events = defaultdict(int)
rb_prev_class = defaultdict(int)
rebound_shot_starts = defaultdict(int)
rebounds = []
rb_count = 0
for game_id, game in games.items():
    last_game_event = None
    for ge in game.game_events:
        if ge.event_type == EventType.Rebound:
            rb_count += 1
            rebounds.append(ge)
            rb_result_counts[ge.result.result_class] += 1
            rb_prev_events[last_game_event.result.result_type] += 1
            rb_prev_class[last_game_event.result.result_class] += 1
            rebound_shot_starts[ge.result.shot_type] += 1
        last_game_event = ge


In [4]:
from build_shot_chance_data import EventType
from collections import defaultdict

rb_result_counts = defaultdict(int)
rb_prev_events = defaultdict(int)
rb_prev_class = defaultdict(int)
rebound_shot_starts = defaultdict(int)
rebounds = []
rb_count = 0
for game_id, game in games.items():
    last_game_event = None
    for ge in game.game_events:
        if ge.event_type == EventType.Rebound:
            rb_count += 1
            rebounds.append(ge)
            rb_result_counts[ge.result.result_class] += 1
            rb_prev_events[last_game_event.result.result_type] += 1
            rb_prev_class[last_game_event.result.result_class] += 1
            rebound_shot_starts[ge.result.shot_type] += 1
        last_game_event = ge

In [5]:
for key, count in rb_result_counts.items():
    print(key, count/rb_count)


ResultClass.DEF_REBOUND 0.6766197136500112
ResultClass.OFF_REBOUND 0.2095268029213985
ResultClass.DEAD_TURNOVER 0.046765535438085704
ResultClass.SAME_TEAM 0.05947841622484512
ResultClass.OTHER_TEAM_FT 0.002696421038701086
ResultClass.JUMPBALL 0.002100892465736429
ResultClass.FT 0.0028122182612219915


In [6]:
for key, count in rb_prev_class.items():
    print(key, count/rb_count)


ResultClass.REBOUND 1.0


In [7]:
for key, count in rb_prev_events.items():
    print(key, count/rb_count)


TryResultType.BLOCKED_SHOT 0.10085110958552866
TryResultType.MISSED_SHOT 0.841945062489144
LiveFreeThrowResult.MISS 0.057203827925327334


In [8]:
for key, count in rebound_shot_starts.items():
    print(key, count/rb_count)


1 0.220461369219444
3 0.20979148228716057
2 0.19584618820357153
7 0.057203827925327334
5 0.07836990595611286
4 0.23832722640838372


# Build rebound chance apm

build rebound chance by shottype

In [9]:
from nba_dataclasses import ResultClass
from build_shot_chance_data import ShotType

reb_by_shottype = {st: {is_blkd: {True: 0, False: 0}
                        for is_blkd in [False, True]} for st in ShotType}
# reb_by_blkd = {is_blkd: {True: 0, False: 0} for is_blkd in [False, True]}
total_rebs = len(rebounds)

for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    is_oreb = rebound.result.result_class in {
        ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
    reb_by_shottype[ShotType(rebound.result.shot_type)
                    ][rebound.result.is_blocked][is_oreb] += 1
    # reb_by_blkd[rebound.result.is_blocked][is_oreb] += 1

for st in ShotType:
    for is_blkd in [False, True]:
        total = reb_by_shottype[st][is_blkd][False] + \
            reb_by_shottype[st][is_blkd][True]
        if total == 0:
            continue
        reb_by_shottype[st][is_blkd][False] = reb_by_shottype[st][is_blkd][False] / total
        reb_by_shottype[st][is_blkd][True] = reb_by_shottype[st][is_blkd][True] / total

reb_by_shottype


{<ShotType.AtRim: 1>: {False: {True: 0.36029926260832124,
   False: 0.6397007373916788},
  True: {True: 0.4172010562052056, False: 0.5827989437947945}},
 <ShotType.ShortMidRange: 2>: {False: {True: 0.28560246563279684,
   False: 0.7143975343672032},
  True: {True: 0.41282375236891977, False: 0.5871762476310802}},
 <ShotType.LongMidRange: 3>: {False: {True: 0.20750070647127689,
   False: 0.7924992935287231},
  True: {True: 0.34275618374558303, False: 0.657243816254417}},
 <ShotType.Arc3: 4>: {False: {True: 0.24870375560538116,
   False: 0.7512962443946188},
  True: {True: 0.41015625, False: 0.58984375}},
 <ShotType.Corner3: 5>: {False: {True: 0.24753747323340472,
   False: 0.7524625267665953},
  True: {True: 0.4918032786885246, False: 0.5081967213114754}},
 <ShotType.FreeThrowNR: 6>: {False: {True: 0, False: 0},
  True: {True: 0, False: 0}},
 <ShotType.FreeThrowR: 7>: {False: {True: 0.115212042263714,
   False: 0.884787957736286},
  True: {True: 0, False: 0}}}

build and solve reb apm

In [10]:
# y = is offenseve rebound chance - chance from shot type
# X = player lineups
# solve for player rebound +/-

from nba_dataclasses import PlayerInfo
import numpy as np
from sklearn import linear_model

player_index = 0
player_indicies: dict[int, PlayerInfo] = {}
lineup_index = 0
lineup_indicies: dict[tuple[tuple[int, int, int, int,
                                  int], tuple[int, int, int, int, int]], int] = {}

for player, ps in nbaTracker.seasons[season_name].player_seasons.items():
    if player not in player_indicies:
        player_indicies[player] = PlayerInfo(player_index)
        player_index += 1
    for team in ps.player_contributions:
        player_indicies[player].teams.add(team)
for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    lineup = rebound.lineup.lineup
    if lineup not in lineup_indicies:
        lineup_indicies[lineup] = lineup_index
        lineup_index += 1
num_lineups = lineup_index
num_players = player_index

y = np.zeros(num_lineups)
ey = np.zeros(num_lineups)
X = np.zeros((num_lineups, num_players*2))
sample_weights = np.zeros(num_lineups)

for rebound in rebounds:
    if rebound.result.result_class == ResultClass.JUMPBALL:
        continue
    lineup = rebound.lineup.lineup
    is_oreb = rebound.result.result_class in {
        ResultClass.OFF_REBOUND, ResultClass.FT, ResultClass.SAME_TEAM}
    oreb_chance = reb_by_shottype[ShotType(rebound.result.shot_type)
                                  ][rebound.result.is_blocked][True]
    y[lineup_indicies[lineup]] += (1 if is_oreb else 0) - oreb_chance
    ey[lineup_indicies[lineup]] += oreb_chance
    sample_weights[lineup_indicies[lineup]] += 1

for lineup, lineup_index in lineup_indicies.items():
    for team_index in [0, 1]:
        for player_index in [0, 1, 2, 3, 4]:
            player_id = lineup[team_index][player_index]
            i = player_indicies[player_id].index + team_index*num_players
            X[lineup_index, i] = 1

reg = linear_model.BayesianRidge()
reg.fit(X, y, sample_weights)

for player in player_indicies.values():
    player.oreb_pm = reg.coef_[player.index]
    player.dreb_pm = reg.coef_[player.index + num_players]


Display best and worst rebounders

In [11]:
max_count = 20
count = 0
print("Top", max_count, "REB_PM")
for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=True):
    name = find_player_by_id(pid)['full_name']
    print(name, player.oreb_pm, player.dreb_pm)
    count += 1
    if count == max_count:
        break
count = 0

print()
print("Bottom", max_count, "REB_PM")
for pid, player in sorted(player_indicies.items(), key=lambda p: p[1].reb_pm, reverse=False):
    name = find_player_by_id(pid)['full_name']
    print(name, player.oreb_pm, player.dreb_pm)
    count += 1
    if count == max_count:
        break


Top 20 REB_PM
Briante Weber 0.4492725284760221 0.060335966133836724
Steven Adams 0.2151893826964091 -0.15052424581813706
Zach Randolph 0.3010836669442734 -0.036267832456417015
Ryan Hollins 0.20950129841306978 -0.11502712870745921
Cristiano Felicio 0.18021016452830613 -0.13504201631584553
Noah Vonleh 0.1673449199841731 -0.1413508603893439
Dwight Howard 0.1879678500511335 -0.10824087801966858
Clint Capela 0.23542551621242028 -0.060258789922858644
Robin Lopez 0.1727087205849283 -0.11751434755578145
Willie Reed 0.1850612810552634 -0.10104373536817868
Lavoy Allen 0.1668373095911683 -0.11894373761894743
Andre Roberson 0.19226429183115162 -0.09252962911154458
Joakim Noah 0.2051026341889192 -0.07874715213897052
Nikola Jokic 0.1378810027206907 -0.14531160862936932
Alex Len 0.16697756302821215 -0.1033118190049501
Tim Duncan 0.10290011084190467 -0.16607825373438534
Jarrett Jack 0.19803616112030487 -0.07027270797498851
Rondae Hollis-Jefferson 0.16490752522474525 -0.09970997424281151
Omer Asik 0.00